In [1]:
suppressPackageStartupMessages({
    require(igraph)
    library(dplyr)
    library(ggvenn)
    library(tidyr)
    library(ggpubr)
})

In [2]:
combine_SSD_WGD <- function(paralogs, ohnologs){
    
    paralogs <- paralogs
    colnames(paralogs) <- c("dup1","dup2")
    paralogs$type <- "SSD"
    
    ohnologs <- ohnologs[,1:2]
    colnames(ohnologs) <- c("dup1","dup2")
    
    index <- c()
    for (i in 1:nrow(ohnologs)){
        dup1 <- ohnologs[i, 1]
        dup2 <- ohnologs[i, 2]
        logic <- which((paralogs$dup1 == dup1 & paralogs$dup2 == dup2)|(paralogs$dup1 == dup2 & paralogs$dup2 == dup1))
        if ( identical(logic, integer(0)) ){
            index <- c(index, i)
        } else {
            paralogs[logic, "type"] <- "WGD"
        }
    }
    tmp <- ohnologs[index, ]
    tmp$type <- "WGD"
    paralogs <- rbind(paralogs, tmp)
    return(paralogs)
    
}

In [3]:
species <- c("Hsap", "Mmus", "Pvit", "Pmar")

SSD_WGD <- Reduce(rbind, lapply(species, FUN = function(s){
    paralogs <- paste0("/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/03.paralog_inferring/",s,".paralogs.txt")
    paralogs <- paralogs <- read.table(paralogs, header = T, sep = "\t")
    ohnologs <- paste0("/mnt/data01/yuanzhen/01.Vertebrate_cell_evo/03.ohnolog_identification/Ohnologs-v2.0/7_FilterOhnologs/",s,".ohnologs.txt")
    ohnologs <- read.delim(ohnologs, header = T)
    res <- combine_SSD_WGD(paralogs, ohnologs)
    res$species <- s
    return(res)
}))

In [4]:
saveRDS(SSD_WGD, file = "Combined.SSD_WGD.rds")
write.table(SSD_WGD, file = "Combined.SSD_WGD.txt", col.names = T, row.names = F, sep = "\t", quote = FALSE)